In [ ]:
# Built-in Libraries
import os
import datetime as dt
from typing import Union

# Third-party Libraries
from dotenv import load_dotenv

# Internal Imports
from mongoapp.models.users import User
from mongoapp.models.transactions import Transaction
from mongoapp.utils.mongodb_connector import MongoConnector, ExpenseTrackerWebAppDB
from mongoapp.utils._password_utils import check_password

load_dotenv()

In [ ]:
expenses_db = ExpenseTrackerWebAppDB(os.getenv('mongodb'))
expenses_db.test_connection()

# Insert/Update User

In [ ]:
username_first = 'dizzydwarfus'
name_first = 'Lian'
email_first = 'liantest@gmail.com'
password_first = 'liantest'
username_second = 'mollysaez'
name_second = 'Molly'
email_second = 'mollytest@gmail.com'
password_second = 'mollytest'

expenses_db.upsert_user(username_first, name_first, email_first, password_first)
expenses_db.upsert_user(username_second, name_second, email_second, password_second)
print(check_password(expenses_db.users.find_one({'_id': username_first})['password'], password_first))
print(check_password(expenses_db.users.find_one({'_id': username_second})['password'], password_second))


In [ ]:
expenses_db.users.find_one({'_id': username_first}),expenses_db.users.find_one({'_id': username_second})

# Upsert Transactions

In [14]:
from mongoapp.models.transactions import Transaction
import datetime as dt


transaction_date = '2023-12-31'
transaction_type = 'expense'
description = 'test transaction - expense'
group = 'testgroup'
paidBy = 'dizzydwarfus'
amount = 100
currency = 'EUR'
split = {
    'dizzydwarfus': 70,
    'mollysaez': 30
}

parsed_date = expenses_db.parse_date(transaction_date)

transaction_id = f'{dt.datetime.strftime(parsed_date, '%Y-%m-%d').replace('-','')}{paidBy}{amount:.2f}{group}'
transaction_id

'20231231dizzydwarfus100.00testgroup'

In [18]:
Transaction(
    transactionDate=expenses_db.parse_date(transaction_date),
    transactionType=transaction_type,
    description=description,
    group=group,
    paidBy=paidBy,
    amount=amount,
    currency=currency,
    split=split,
    transactionId=transaction_id,
)

20231231dizzydwarfus100.00testgroup


Transaction(transactionDate=datetime.datetime(2023, 12, 31, 0, 0), transactionType='expense', description='test transaction - expense', group='testgroup', paidBy='dizzydwarfus', amount=100.0, currency='EUR', split={'dizzydwarfus': 70.0, 'mollysaez': 30.0}, transactionId='20231231dizzydwarfus100.00testgroup')

In [16]:
expenses_db.upsert_transaction(\
    transaction_date=transaction_date,
    transaction_type=transaction_type,
    description=description,
    group=group,
    paidBy=paidBy,
    amount=amount,
    currency=currency,
    split=split
    )

20231231dizzydwarfus100.00testgroup


True

# Find Transactions

In [17]:
expenses_db.transactions.find_one({'transactionDate': {'$gt': dt.datetime(2021,1,1)}})


{'_id': '20231231dizzydwarfus100.00testgroup',
 'amount': 100.0,
 'createdDate': datetime.datetime(2024, 1, 28, 16, 1, 3, 341000),
 'currency': 'EUR',
 'description': 'test transaction - expense',
 'group': 'testgroup',
 'paidBy': 'dizzydwarfus',
 'split': {'dizzydwarfus': 70.0, 'mollysaez': 30.0},
 'transactionDate': datetime.datetime(2023, 12, 31, 0, 0),
 'transactionId': '20231231dizzydwarfus100.00testgroup',
 'transactionType': 'expense',
 'updatedDate': datetime.datetime(2024, 1, 28, 16, 11, 47, 870000)}